In [2]:
from jupyterscad import view
from  solid2 import * 

In [54]:
debug = True
render = True
name = "body-refactor"
set_global_fn(200)

# define base dimensions
# ring size
base_r = 14
# height of the ring
base_d = 4
# width of ring material
base_w = 1
# gap between inner and outer circle
gap = 1
# manufacturing tolerance
tolerance = 0
cut = 0.5
mag_count = 19
base_inner_r = base_r-base_w
mag_a = 1
mag_b = base_d
mag_cut_a = mag_a + cut
mag_cut_b = mag_b + 2

# minimal width of material for cutouts etc.
material_min = 0.2

inner_r,inner_d,inner_w = [base_r,base_d,base_w]
outer_r,outer_d,outer_w = [inner_r+inner_w+gap,inner_d,inner_w]
#if outer_<= mag_a:
outer_r += material_min
outer_w += material_min


In [63]:

obj = []
inner_base = cylinder(
    inner_d,
    inner_r+inner_w,
    inner_r+inner_w
)
inner_cut = cylinder(
    inner_d+2,
    inner_r,
    inner_r
)
outer_base = cylinder(
    outer_d,
    outer_r+outer_w,
    outer_r+outer_w
)
outer_cut = cylinder(
    outer_d+2,
    outer_r,
    outer_r
)


magnets = []
magnet_cuts = []

mag_base = cube(mag_a,mag_a,mag_b).translate(
    outer_r,-mag_a/2,0
).translate(
    -cut/2,0,0
)
mag_cut = cube(mag_cut_a,mag_cut_a,mag_cut_b).translate(
    outer_r,-mag_cut_a/2,-1
).translate(
    outer_w-cut-gap-material_min,0,0
)

for magnet in range(0, mag_count):
    magnets += mag_base.rotate(0,0,magnet*360/mag_count)
    magnet_cuts += mag_cut.rotate(0,0,magnet*360/mag_count)
    
inner = inner_base - inner_cut.translate(0,0,-1)
outer = outer_base - outer_cut.translate(0,0,-1) - magnet_cuts
#outer = outer_base - outer_cut.translate(0,0,-1)
   

magnets = magnets.color([0.9,0,0])
inner = inner.color([1,0.1,0.5])
outer = outer.color([0,0.9,0])
rendered = [ inner,outer,magnets]
#obj += inner +outer - magnet_cuts + magnets

for i, r in enumerate(rendered):
    if debug:
        obj += r.translate(0,0,-2*i)
    else:
        obj += r.translate(0,0,0*i)



if render:
    obj.save_as_scad(f"{name}.scad")
    obj.save_as_stl(f"{name}.stl")


SyntaxError: invalid syntax (1628068219.py, line 30)